In [1]:
from pycocotools.coco import COCO # COCO python library
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

import random
import string

import cv2
import os
from pickle import dump, load
import json

import nltk
from nltk.corpus import stopwords

import tensorflow as tf
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Input, Dropout, Attention, add
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.optimizers import Adam
from keras_applications.xception import Xception, preprocess_input
from keras_preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
from keras.models import Model, load_model


In [4]:
data = r"data\input\trainval\instances_train2017.json"
coco = COCO(data)

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: 'data\\input\\trainval\\instances_train2017.json'

In [2]:
#Identificar Categorias

cats = coco.loadCats(coco.getCatIds())
maincategories = list(set([cat['supercategory'] for cat in cats]))

print("Número de categorías principales: ", len(maincategories))
print("Lista de categorías principales: ", maincategories)

NameError: name 'coco' is not defined

In [ ]:
#Identificar Sub-Categorías

subcategories = [cat['name'] for cat in cats]

print("Number of sub categories: ", len(subcategories))
print("List of sub categories: ", subcategories)

In [ ]:
#Sub-categorías con su correspondientes IDs

catIds = coco.getCatIds(catNms=subcategories)

subcategories_Ids = dict()
for i in range(0,len(subcategories)):
    subcategories_Ids[subcategories[i]] = catIds[i]

print("Sub categorías con IDs :",subcategories_Ids)

In [ ]:
#Total de imágenes para cada sub categoría

subcategories_imageIds = dict()

for i in range(0,len(catIds)):
    imgIds = coco.getImgIds(catIds=catIds[i])
    img = []
    for j in imgIds:
        img.append(j)
    subcategories_imageIds[subcategories[i]] = img

print("Sub-categorias con Image IDs :",len(subcategories_imageIds))

In [ ]:
length_dict = {key: len(value) for key, value in subcategories_imageIds.items()}
print("Imágenes totales en cada sub categoría: ", length_dict)

In [ ]:
#Se seleccionan dos subcategorías Bicicleta y Avión para llevar a cabo 
#el proyecto generador de leyendas de imágenes

train_cats = subcategories_imageIds['bicycle'] + subcategories_imageIds['airplane']
imgIdss = coco.getImgIds(imgIds = train_cats)
print("Total Images: ", len(imgIdss))

In [ ]:
#Cargamos algunas imagenes aleatorias

fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix = imgIdss
random.shuffle(next_pix)

for i, img_path in enumerate(next_pix[0:12]):

    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')

    img = coco.loadImgs(img_path)[0]
    I = io.imread(img['coco_url'])
    plt.imshow(I)

plt.show()

In [ ]:
#Cargamos esas imagenes con objetos segmentados en ellas

fig = plt.gcf()
fig.set_size_inches(16, 16)

for i, img_path in enumerate(next_pix[0:12]):

    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')

    img = coco.loadImgs(img_path)[0]
    I = io.imread(img['coco_url'])
    plt.imshow(I)
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    # print(anns)
    coco.showAnns(anns)


plt.show()

In [ ]:
# Cargamos las imagenes con keypoints en ellas

annFile = r"data\input\trainval\person_keypoints_train2017.json"
coco_kps=COCO(annFile)

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

for i, img_path in enumerate(next_pix[0:12]):
    
    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')

    img = coco.loadImgs(img_path)[0]
    I = io.imread(img['coco_url'])
    plt.imshow(I)
    annIds = coco_kps.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco_kps.loadAnns(annIds)
    coco_kps.showAnns(anns)
    

plt.show()

In [ ]:
# Cargamos las imágenes con sus correspondientes subtítulos
annFile = "data\input\trainval\captions_train2017.json"
coco_caps=COCO(annFile)

In [ ]:
img = coco.loadImgs(next_pix[0])[0]
I = io.imread(img['coco_url'])
plt.imshow(I)
annIds = coco_caps.getAnnIds(imgIds=img['id']);
anns = coco_caps.loadAnns(annIds)
coco_caps.showAnns(anns)
plt.show()

In [ ]:
img = coco.loadImgs(next_pix[1])[0]
I = io.imread(img['coco_url'])
plt.imshow(I)
annIds = coco_caps.getAnnIds(imgIds=img['id']);
anns = coco_caps.loadAnns(annIds)
coco_caps.showAnns(anns)
plt.show()

In [ ]:
img = coco.loadImgs(next_pix[10])[0]
I = io.imread(img['coco_url'])
plt.imshow(I)
annIds = coco_caps.getAnnIds(imgIds=img['id']);
anns = coco_caps.loadAnns(annIds)
coco_caps.showAnns(anns)
plt.show()